<a href="https://colab.research.google.com/github/sophryu99/embedding-word2vec/blob/main/fastText/word2vec_fastText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word2Vec vs FastText

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

In [3]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")


('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7fac0ae7b3d0>)

## 전처리

In [4]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [8]:
print('총 샘플의 개수 : {}'.format(len(result)))


총 샘플의 개수 : 273424


In [9]:
# 샘플 3개만 출력
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


## Word2Vec - Training

- size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
- window = 컨텍스트 윈도우 크기
- min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
- workers = 학습을 위한 프로세스 수
- sg = 0은 CBOW, 1은 Skip-gram.



In [10]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

In [11]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8326016664505005), ('guy', 0.8062232136726379), ('lady', 0.7660772800445557), ('boy', 0.7640954852104187), ('girl', 0.7564641833305359), ('gentleman', 0.7490572929382324), ('soldier', 0.7443642616271973), ('kid', 0.7041044235229492), ('friend', 0.6828451752662659), ('poet', 0.6627794504165649)]


In [12]:
# 모델 저장
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [13]:
model.wv.most_similar("woman")

[('girl', 0.8355351686477661),
 ('man', 0.8326016664505005),
 ('boy', 0.7908482551574707),
 ('lady', 0.7778284549713135),
 ('child', 0.7594408988952637),
 ('soldier', 0.7497005462646484),
 ('kid', 0.7463718056678772),
 ('gentleman', 0.7348035573959351),
 ('guy', 0.6991581916809082),
 ('person', 0.6785804629325867)]

In [14]:
# Word2Vec는 학습 데이터에 존재하지 않는 단어. 즉, 모르는 단어에 대해서는 임베딩 벡터가 존재하지 않기 때문에 단어의 유사도를 계산할 수 없다
model.wv.most_similar("electrofishing")

KeyError: ignored

## fastText - Training

In [15]:
from gensim.models import FastText
model = FastText(result, size=100, window=5, min_count=5, workers=4, sg=0)

In [16]:
model.wv.most_similar("electrofishing")


[('electric', 0.8987501859664917),
 ('electro', 0.8937985301017761),
 ('electrolux', 0.8846410512924194),
 ('electron', 0.8832679986953735),
 ('electronic', 0.8715877532958984),
 ('electromagnet', 0.8704180717468262),
 ('electroshock', 0.8703902959823608),
 ('electrolyte', 0.8653165102005005),
 ('electrode', 0.858948826789856),
 ('electrical', 0.8520902395248413)]